In [1]:
import pandas as pd
import numpy as np 
import csv
import re
import string
import time

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings("ignore")

In [2]:
start = time.time()
path = '/home/manikya_varshney/Documents/Python/Yale/h01-20200818-10files/h01-20200818-153021.csv'
#data = pd.read_csv(path, index_col=None, header=0, engine='python' )
data = pd.read_csv(path, low_memory=False)
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  0.6255223751068115 sec


In [3]:
data.shape

(19992, 79)

In [4]:
cols = [0, 1, 3, 4, 5, 6, 7, 8, 9,
        10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        22, 29,
        30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
        40, 41, 42, 43, 45, 
        50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
        60, 61, 62, 63, 64, 65, 67,
        72, 73, 74, 75, 76, 77]

data = data[data.columns.values[cols]]

In [5]:
data.shape

(19992, 59)

In [6]:
data_en = data[data['lang'] == 'en'].reset_index(drop=True)

In [7]:
data_en.shape

(11229, 59)

In [8]:
data_en['RT_Flag'] = data_en['text'].str[:2]=='RT'

In [9]:
data_en.shape

(11229, 60)

In [10]:
data_en = data_en.drop_duplicates()

##### Merge text and full text columns

In [11]:
data_en['extended_tweet_full_text'] = np.where(~data_en['truncated'], data_en['text'], data_en['extended_tweet_full_text'])

In [12]:
data_en['RT_full_text'] = np.where(data_en['RT_full_text'].isna(),data_en['RT_text'],data_en['RT_full_text'])

In [13]:
data_en['QT_full_text'] = np.where(data_en['QT_full_text'].isna(),data_en['QT_text'],data_en['QT_full_text'])

In [14]:
data_en.shape

(11229, 60)

In [15]:
data_en['extended_tweet_full_text']

0        can we wear clone helmets instead of masks to ...
1        RT @PhilipWegmann: Biden thanks Cuomo. The AP ...
2        The economic recovery that's unfolding before ...
3        This is where we launch the Little River Clean...
4        RT @_behindherlense: Nothing that includes men...
                               ...                        
11224    Covid is raging in Florida, and I just got a c...
11225    I thought I’d like working from home for these...
11226    RT @_Dumile: Lessons from covid:\n\nLife is an...
11227    RT @abbygov: it’s a good thing tattoo parlors ...
11228                I—-we ain’t never goin off quarantine
Name: extended_tweet_full_text, Length: 11229, dtype: object

##### Data Cleaning

In [16]:
import pandas as pd
import numpy as np 
import csv
import re
import string
import emoji

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import warnings
warnings.filterwarnings("ignore")

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])
#demoji.download_codes()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
#Convert to lower case
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].str.lower()
data_en['RT_full_text'] = data_en['RT_full_text'].str.lower()
data_en['QT_full_text'] = data_en['QT_full_text'].str.lower()

In [18]:
data_en['extended_tweet_full_text']

0        can we wear clone helmets instead of masks to ...
1        rt @philipwegmann: biden thanks cuomo. the ap ...
2        the economic recovery that's unfolding before ...
3        this is where we launch the little river clean...
4        rt @_behindherlense: nothing that includes men...
                               ...                        
11224    covid is raging in florida, and i just got a c...
11225    i thought i’d like working from home for these...
11226    rt @_dumile: lessons from covid:\n\nlife is an...
11227    rt @abbygov: it’s a good thing tattoo parlors ...
11228                i—-we ain’t never goin off quarantine
Name: extended_tweet_full_text, Length: 11229, dtype: object

In [19]:
#Removing emojis
def demoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00010000-\U0010ffff"
                               "]+", flags=re.UNICODE)
    return(emoji_pattern.sub(r'', text))

data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].astype(str)
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].apply(lambda x:demoji(x))

data_en['RT_full_text'] = data_en['RT_full_text'].astype(str)
data_en['RT_full_text'] = data_en['RT_full_text'].apply(lambda x:demoji(x))

data_en['QT_full_text'] = data_en['QT_full_text'].astype(str)
data_en['QT_full_text'] = data_en['QT_full_text'].apply(lambda x:demoji(x))

In [20]:
data_en['extended_tweet_full_text']

0        can we wear clone helmets instead of masks to ...
1        rt @philipwegmann: biden thanks cuomo. the ap ...
2        the economic recovery that's unfolding before ...
3        this is where we launch the little river clean...
4        rt @_behindherlense: nothing that includes men...
                               ...                        
11224    covid is raging in florida, and i just got a c...
11225    i thought i’d like working from home for these...
11226    rt @_dumile: lessons from covid:\n\nlife is an...
11227    rt @abbygov: it’s a good thing tattoo parlors ...
11228                i—-we ain’t never goin off quarantine
Name: extended_tweet_full_text, Length: 11229, dtype: object

In [21]:
#Remove URLs
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].str.replace(r"http\S+| www\S+| https\S+| \S+\.com\S+| \S+\.com", "", regex=True)

data_en['RT_full_text'] = data_en['RT_full_text'].str.replace(r"http\S+| www\S+| https\S+| \S+\.com\S+| \S+\.com", "", regex=True)

data_en['QT_full_text'] = data_en['QT_full_text'].str.replace(r"http\S+| www\S+| https\S+| \S+\.com\S+| \S+\.com", "", regex=True)

In [22]:
data_en['extended_tweet_full_text']

0        can we wear clone helmets instead of masks to ...
1        rt @philipwegmann: biden thanks cuomo. the ap ...
2        the economic recovery that's unfolding before ...
3        this is where we launch the little river clean...
4        rt @_behindherlense: nothing that includes men...
                               ...                        
11224    covid is raging in florida, and i just got a c...
11225    i thought i’d like working from home for these...
11226    rt @_dumile: lessons from covid:\n\nlife is an...
11227    rt @abbygov: it’s a good thing tattoo parlors ...
11228                i—-we ain’t never goin off quarantine
Name: extended_tweet_full_text, Length: 11229, dtype: object

In [23]:
#Remove user @
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].str.replace(r'\@[\w]+', "", regex=True)

data_en['RT_full_text'] = data_en['RT_full_text'].str.replace(r'\@[\w]+', "", regex=True)

data_en['QT_full_text'] = data_en['QT_full_text'].str.replace(r'\@[\w]+', "", regex=True)

In [24]:
data_en['extended_tweet_full_text']

0        can we wear clone helmets instead of masks to ...
1        rt : biden thanks cuomo. the ap recently estim...
2        the economic recovery that's unfolding before ...
3        this is where we launch the little river clean...
4        rt : nothing that includes men at night is wor...
                               ...                        
11224    covid is raging in florida, and i just got a c...
11225    i thought i’d like working from home for these...
11226    rt : lessons from covid:\n\nlife is an absolut...
11227    rt : it’s a good thing tattoo parlors are clos...
11228                i—-we ain’t never goin off quarantine
Name: extended_tweet_full_text, Length: 11229, dtype: object

In [25]:
#Remove punctuations
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].str.translate(str.maketrans("", "", string.punctuation))

data_en['RT_full_text'] = data_en['RT_full_text'].str.translate(str.maketrans("", "", string.punctuation))

data_en['QT_full_text'] = data_en['QT_full_text'].str.translate(str.maketrans("", "", string.punctuation))

In [26]:
data_en['extended_tweet_full_text']

0        can we wear clone helmets instead of masks to ...
1        rt  biden thanks cuomo the ap recently estimat...
2        the economic recovery thats unfolding before u...
3        this is where we launch the little river clean...
4        rt  nothing that includes men at night is wort...
                               ...                        
11224    covid is raging in florida and i just got a co...
11225    i thought i’d like working from home for these...
11226    rt  lessons from covid\n\nlife is an absolute ...
11227    rt  it’s a good thing tattoo parlors are close...
11228                 i—we ain’t never goin off quarantine
Name: extended_tweet_full_text, Length: 11229, dtype: object

In [27]:
#More Cleaning
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ')\
          .str.replace('(http\S+)', ' ')\
          .str.replace('([^0-9a-z \t])',' ')\
          .str.replace(' +',' ')

data_en['RT_full_text'] = data_en['RT_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')
data_en['RT_full_text'] = data_en['RT_full_text'].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ')\
          .str.replace('(http\S+)', ' ')\
          .str.replace('([^0-9a-z \t])',' ')\
          .str.replace(' +',' ')

data_en['QT_full_text'] = data_en['QT_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')
data_en['QT_full_text'] = data_en['QT_full_text'].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ')\
          .str.replace('(http\S+)', ' ')\
          .str.replace('([^0-9a-z \t])',' ')\
          .str.replace(' +',' ')

In [28]:
data_en['extended_tweet_full_text']

0        can we wear clone helmets instead of masks to ...
1        rt biden thanks cuomo the ap recently estimate...
2        the economic recovery thats unfolding before u...
3        this is where we launch the little river clean...
4        rt nothing that includes men at night is worth it
                               ...                        
11224    covid is raging in florida and i just got a co...
11225    i thought i d like working from home for these...
11226    rt lessons from covid life is an absolute gift...
11227    rt it s a good thing tattoo parlors are closed...
11228                 i we ain t never goin off quarantine
Name: extended_tweet_full_text, Length: 11229, dtype: object

In [29]:
#Tokenizing
data_en.extended_tweet_full_text = data_en.extended_tweet_full_text.astype(str)
data_en['extended_tweet_full_text'] = data_en.apply(lambda row: nltk.word_tokenize(row.extended_tweet_full_text), axis=1)


data_en.RT_full_text = data_en.RT_full_text.astype(str)
data_en['RT_full_text'] = data_en.apply(lambda row: nltk.word_tokenize(row.RT_full_text), axis=1)


data_en.QT_full_text = data_en.QT_full_text.astype(str)
data_en['QT_full_text'] = data_en.apply(lambda row: nltk.word_tokenize(row.QT_full_text), axis=1)

# remove stopwords
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].apply(lambda x: [word for word in x if word not in stop_words])

data_en['RT_full_text'] = data_en['RT_full_text'].apply(lambda x: [word for word in x if word not in stop_words])

data_en['QT_full_text'] = data_en['QT_full_text'].apply(lambda x: [word for word in x if word not in stop_words])

In [30]:
data_en['extended_tweet_full_text'][26]

['due',
 'to',
 'covid19',
 'consumers',
 'initiating',
 'startling',
 'number',
 'chargebacks',
 'improve',
 'your',
 'companys',
 'defense',
 'unwarranted',
 'refund',
 'requests',
 'clicking',
 'the',
 'link',
 'chargebacks']

In [31]:
#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [32]:
#Lemmatizing
lemmatizer = WordNetLemmatizer()
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].apply(lambda x: [lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in x])

data_en['RT_full_text'] = data_en['RT_full_text'].apply(lambda x: [lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in x])

data_en['QT_full_text'] = data_en['QT_full_text'].apply(lambda x: [lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in x])

In [33]:
data_en['extended_tweet_full_text'][26]

['due',
 'to',
 'covid19',
 'consumer',
 'initiate',
 'startle',
 'number',
 'chargebacks',
 'improve',
 'your',
 'company',
 'defense',
 'unwarranted',
 'refund',
 'request',
 'click',
 'the',
 'link',
 'chargebacks']

In [34]:
#Joining the lemmetized tokens to form string
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].apply(lambda x: " ".join([word for word in x]))

data_en['RT_full_text'] = data_en['RT_full_text'].apply(lambda x: " ".join([word for word in x]))

data_en['QT_full_text'] = data_en['QT_full_text'].apply(lambda x: " ".join([word for word in x]))

In [35]:
data_en['extended_tweet_full_text'][26]

'due to covid19 consumer initiate startle number chargebacks improve your company defense unwarranted refund request click the link chargebacks'

In [36]:
#Remove punctuations
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].str.translate(str.maketrans("", "", string.punctuation))
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].str.replace("’", '').str.replace("“", '').str.replace("”", '')
data_en['extended_tweet_full_text'] = data_en['extended_tweet_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')

data_en['RT_full_text'] = data_en['RT_full_text'].str.translate(str.maketrans("", "", string.punctuation))
data_en['RT_full_text'] = data_en['RT_full_text'].str.replace("’", '').str.replace("“", '').str.replace("”", '')
data_en['RT_full_text'] = data_en['RT_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')

data_en['QT_full_text'] = data_en['QT_full_text'].str.translate(str.maketrans("", "", string.punctuation))
data_en['QT_full_text'] = data_en['QT_full_text'].str.replace("’", '').str.replace("“", '').str.replace("”", '')
data_en['QT_full_text'] = data_en['QT_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')

In [37]:
data_en['extended_tweet_full_text']

0                 wear clone helmet instead mask to school
1        rt biden thanks cuomo the ap recently estimate...
2        the economic recovery thats unfold u encourage...
3        launch the little river cleanup last time clea...
4                    rt nothing include men at night worth
                               ...                        
11224    covid rag florida get cold call from someone b...
11225    thought like work from home next two week day ...
11226    rt lesson from covid life absolute gift job tr...
11227    rt good thing tattoo parlor close covid otherw...
11228                                never goin quarantine
Name: extended_tweet_full_text, Length: 11229, dtype: object

In [38]:
data_en.shape

(11229, 60)

In [39]:
sample = data_en[['user_id', 'id','extended_tweet_full_text', 'RT_user_id', 'RT_id','RT_full_text',
                  'QT_user_id', 'QT_id','QT_full_text',]]

In [40]:
sample.to_csv('/home/manikya_varshney/Documents/Python/Yale/h01-20200818-10files/check.csv',index=False)